In [1]:
# Interestingly Logistic Regression in performing well on the data with TF-IDF Vectorizer.
# There were many big models like XgBoost , RF , GB but Logistic Regression is performing to well.
# For this moment of time we are focusing on learning to integrate the concepts learned in workflow so not focusing on accuracy much.
# I am taking Logistic as my model now for final experimentation.

In [2]:
import mlflow
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
import os
import dagshub


In [6]:
df = pd.read_csv('https://raw.githubusercontent.com/campusx-official/jupyter-masterclass/main/tweet_emotions.csv').drop(columns=['tweet_id'])


In [7]:
# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['content'] = df['content'].apply(lower_case)
        df['content'] = df['content'].apply(remove_stop_words)
        df['content'] = df['content'].apply(removing_numbers)
        df['content'] = df['content'].apply(removing_punctuations)
        df['content'] = df['content'].apply(removing_urls)
        df['content'] = df['content'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise


In [8]:
df = normalize_text(df)

In [9]:
x = df['sentiment'].isin(['happiness','sadness'])
df = df[x]

In [10]:
df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})

In [11]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['content'])
y = df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
mlflow.set_tracking_uri("https://dagshub.com/AJINKYA-TEMGIRE/Emotion_Detection.mlflow")
dagshub.init(repo_owner='AJINKYA-TEMGIRE', repo_name='Emotion_Detection', mlflow=True)

mlflow.set_experiment("Logistic_Regression_Hyperparameter")

Accessing as AJINKYA-TEMGIRE

Initialized MLflow to track repo "AJINKYA-TEMGIRE/Emotion_Detection"

Repository AJINKYA-TEMGIRE/Emotion_Detection initialized!

2024/08/04 11:56:26 INFO mlflow.tracking.fluent: Experiment with name 'Logistic_Regression_Hyperparameter' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/712b32bc8b8d4c80880c4091fe90573d', creation_time=1722752788095, experiment_id='4', last_update_time=1722752788095, lifecycle_stage='active', name='Logistic_Regression_Hyperparameter', tags={}>

In [13]:
param_grid = {
    'C': [0.1, 1, 10],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear']
}


In [14]:
with mlflow.start_run():

    # Perform grid search
    grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5, scoring='f1', n_jobs=-1)
    grid_search.fit(X_train, y_train)

    # Log each parameter combination as a child run
    for params, mean_score, std_score in zip(grid_search.cv_results_['params'], grid_search.cv_results_['mean_test_score'], grid_search.cv_results_['std_test_score']):
        with mlflow.start_run(run_name=f"LR with params: {params}", nested=True):
            model = LogisticRegression(**params)
            model.fit(X_train, y_train)
            
            # Model evaluation
            y_pred = model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred)
            recall = recall_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred)
            
            # Log parameters and metrics
            mlflow.log_params(params)
            mlflow.log_metric("mean_cv_score", mean_score)
            mlflow.log_metric("std_cv_score", std_score)
            mlflow.log_metric("accuracy", accuracy)
            mlflow.log_metric("precision", precision)
            mlflow.log_metric("recall", recall)
            mlflow.log_metric("f1_score", f1)
            
            
            # Print the results for verification
            print(f"Mean CV Score: {mean_score}, Std CV Score: {std_score}")
            print(f"Accuracy: {accuracy}")
            print(f"Precision: {precision}")
            print(f"Recall: {recall}")
            print(f"F1 Score: {f1}")

    # Log the best run details in the parent run
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_
    mlflow.log_params(best_params)
    mlflow.log_metric("best_f1_score", best_score)
    
    print(f"Best Params: {best_params}")
    print(f"Best F1 Score: {best_score}")

    # Save and log the notebook
    notebook_path = "best_model_hyper.ipynb"
    os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
    mlflow.log_artifact(notebook_path)

    # Log Input Data
    dataset = mlflow.data.from_numpy(X.todense())
    mlflow.log_input(dataset , "dataset")

    # Log model
    mlflow.sklearn.log_model(grid_search.best_estimator_, "model")

Mean CV Score: 0.5400565011817193, Std CV Score: 0.02535032729518983
Accuracy: 0.6819277108433734
Precision: 0.7963272120200334
Recall: 0.46995073891625616
F1 Score: 0.5910780669144982
Mean CV Score: 0.7780911786800282, Std CV Score: 0.005264604559330897
Accuracy: 0.7783132530120482
Precision: 0.7655502392344498
Recall: 0.7881773399014779
F1 Score: 0.7766990291262137
Mean CV Score: 0.778928244221831, Std CV Score: 0.014315613804664937
Accuracy: 0.7816867469879518
Precision: 0.7486725663716814
Recall: 0.8334975369458129
F1 Score: 0.788811188811189
Mean CV Score: 0.7915625393845699, Std CV Score: 0.007069118975692473
Accuracy: 0.7956626506024096
Precision: 0.7800947867298578
Recall: 0.8108374384236453
F1 Score: 0.795169082125604


c:\Python 310\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Mean CV Score: 0.7836909677583195, Std CV Score: 0.007778118741074336
Accuracy: 0.7855421686746988
Precision: 0.7658582089552238
Recall: 0.8088669950738916
F1 Score: 0.7867752755150935
Mean CV Score: 0.7935424737640171, Std CV Score: 0.007546482282203498
Accuracy: 0.7937349397590362
Precision: 0.7787274453941121
Recall: 0.8078817733990148
F1 Score: 0.793036750483559
Best Params: {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}
Best F1 Score: 0.7935424737640171


c:\Python 310\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
